## <font color=#0099CC>mIAx - Taller Renta Fija - ANÁLISIS CARTERA DE RENTA FIJA</font>

En esta práctica, desarrollaremos un análisis relativamente exahustivo de un universo de Renta Fija, en concreto, de bonos corporativos. Además, construiremos y analizaremos varias carteras.

Para ello, contaremos con la siguiente información almacenada en la carpeta *data*:
- Universo de bonos, con sus características esenciales (fichero *universo.csv*)
- Histórico de precios de cierre del universo de bonos anterior (fichero *precios_historicos_universo.csv*)
- Curva de tipos de interés €STR (fichero *curvaESTR.csv*)
- Histórico de precios de otros índices que nos serán de utilidad (fichero *precios_historicos_varios*):
    - Índices de crédito: ITRAXX Main y ITRAXX XOVER. Serán útiles para la cobertura del riesgo de crédito.
    - Futuros sobre el *Schatz* (DU1), *BOBL* (OE1) y *BUND* (RX1). Serán útiles para la cobertura de los tipos de interés.
    - Índice de crédito *RECMTREU*, que valdría como benchmark de las carteras que construyamos.

No necesariamente se usará toda toda la información

En líneas generales, estos son los ejercicios que completaremos, aunque los detallaremos más en cada apartado:
1. Análisis de datos. En esta sección, haremos un análisis de la información que tenemos de cada bono y lo que significa. Asimismo, haremos los tratamientos y limpieza que necesitemos para luego poder usarlos.
2. Valoración de los bonos del universo utilizando la curva de descuento y bajo ciertas asunciones. Comparación de estos precios con los precios de mercado.
3. Cálculo del spread que pagan los bonos sobre la curva.
4. Cálculo de *yield*, duración y convexidad.
5. Contrucción de una cartera equiponderada con todos los bonos del universo. Contraste con el benchmark (os proponemos el índice RECMTREU para el que os hemos dado los precios) y backtest de la estrategia. ¡OJO! El índice es *Total Return*.
6. Tienes el mandato de construir una cartera de como máximo **20** bonos corporativos con ese universo y una serie de restricciones y, claro, maximizando la rentabilidad total de la cartera:
    - La duración de la cartera no debe superar los 3 años
    - La exposición a emisiones HY no puede superar el 10% de la cartera
    - No puedes invertir en deuda subordinada
    - No se puede invertir en emisiones de tamaño igual o inferior a 500 millones
    - No se puede invertir más de un 10% del capital en una misma emisión
    - No puede haber más de un 15% de concentración en un mismo emisor
    (¡OJO! No estamos teniendo en cuenta en este ejercicio si hubiera un mínimo de inversión, lo cuál sería un dato relevante tener en cuenta en un caso real)

    6.1. Constrúyela a fecha de hoy

    6.2. Teniendo en cuenta la naturaleza que nos están pidiendo para la cartera, ¿añadirías alguna otra restricción?

    6.3. ¿Cómo medirías el riesgo de crédito de la cartera?

    6.4. ¿Cómo medirías el riesgo de liquidez de la cartera?

    6.5. Describe cómo habría que hacer el backtest de esta cartera
7. Ahora, se te pide que cubras la exposición de la cartera a los tipos de interés. Con la información que tienes, ¿cómo lo harías?
8. ¿Y si quisieras cubrir total o parcialmente el riesgo de crédito? Usa de nuevo la información que tienes.
9. ¿Cómo construirías tu cartera? ¿Se te ocurre alguna estrategia específica, por ejemplo, de valor relativo?


📣 <font color=#CC6600>**¡NORMAS!**</font>

La práctica se puede hacer en grupos de hasta **3 personas** y deberá entregarse antes del **27 de noviembre**.

Cada grupo expondrá una parte de los ejercicios en la clase del día 27, donde la resolveremos juntos a modo de taller. Esta exposición contará hasta **1 punto** de la nota final.

Además, se valorará positivamente para la práctica la participación en las clases.

📣 <font color=#CC6600>**¡IMPORTANTE!**</font>

Todo el código implementado debe estar debidamente comentado e incluir conclusiones de los resultados obtenidos para optar a la máxima puntuación. Asimismo, se debe responder a las preguntas planteadas.

Las propuestas de mejora o posibles trabajos futuros se valorarán positivamente también.

Usa las fuentes de información que consideres necesarias para apoyar tus respuestas.

### <font color=#336699>Librerías</font>

In [ ]:
from importlib import resources
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import seaborn as sns
from dateutil.relativedelta import relativedelta
from scipy import optimize
from scipy.optimize import minimize


In [ ]:
# Estilo
#plt.style.use('dark_background')
plt.style.use('default')

In [ ]:
# Fecha de análisis
fecha_analisis = pd.to_datetime('2025-10-01')

### <font color=#336699>1. Datos</font>

<style>.white {background-color: #595959}

</style><div class="gray">❕💬¿Qué observas en los datos? Analiza la información que tenemos del universo</div>

Haz un primer análisis visual de la información que tenemos del universo de bono.
Entre otras, plantéate cuestiones como:
- ¿Divisas?
- ¿Tipo de bonos? ¿Fijo/Flotante? ¿Prelación? ¿Opcionalidad? ¿Hay bonos perpetuos?
- ¿Sectores? ¿Emisores? Si invirtéramos en todos los bonos, ¿dirías a priori que la cartera está diversificada?
- ¿Ratings? (Riesgo de crédito)
- ¿Otros datos cuantitativos?
    - Riesgo de liquidez - Horquillas y nominal vivo
- ¿Hay *gaps* en la información que vamos a tener que tratar?

In [ ]:
from pathlib import Path
# Ruta relativa desde src/ hacia data/ (un nivel arriba)
# Si el notebook se ejecuta desde src/, esta ruta funcionará
universo_path = Path('../data/universo.csv')
universo = pd.read_csv(universo_path, sep=';', encoding='utf-8-sig')

Lo primero que nos gustaría destacar, que no tenemos el nominal de cada bono en el archivo *universo.csv*, por lo que asumimos que el valor nominal será de **100 €** para cada bono. Además, hemos comprobado que la columna *Price* del archivo *universo.csv* coincide con la última columna del archivo *precios_historicos_universo.csv*, con fecha 01/10/2025.

#### <font color=#808080>Divisas</font>

In [ ]:
print("ANÁLISIS DE DIVISAS:")
currency = universo['Ccy'].value_counts(dropna = False) 
for curr, quantity in currency.items():
    print(f"  - {curr}: {quantity} bonos")

**Conclusión**: Todos los bonos están denominados en EUR. Esto elimina el riesgo por cambio de divisas.

In [ ]:
# Análisis de tipos de bonos
print(" TIPOS DE BONOS:")

# Tipo de cupón
print("Tipo de Cupón:")
coupon_types = universo['Coupon Type'].value_counts(dropna = False) 
for coupon, quantity in coupon_types.items():
    print(f"  - {coupon}: {quantity} bonos")

# Visualización
fig, axes = plt.subplots(2, 3, figsize=(14, 10))

# Gráfico 1: Tipo de cupón
coupon_types.plot(kind='pie', ax=axes[0,0], autopct='%1.1f%%', colors=['#4CAF50', '#FF9800'])
axes[0,0].set_title('Distribución por Tipo de Cupón')
axes[0,0].set_ylabel('')

# Prelación
print("Prelación (Seniority):")
seniority = universo['Seniority'].value_counts(dropna = False) 
for senior, quantity in seniority.items():
    print(f"  - {senior}: {quantity} bonos")

# Gráfico 2: Prelación
seniority.plot(kind='barh', ax=axes[0,1], color='#2196F3')
axes[0,1].set_title('Tipos de Prelación')
axes[0,1].set_xlabel('Número de Bonos')

# Callable
print("Optionalidad (Callable):")
callable_analysis = universo['Callable'].value_counts(dropna = False) 

for call , quantity in callable_analysis.items():
    print(f"  - {call}: {quantity} bonos")

callable_pct = callable_analysis.get('Y', 0)/len(universo)*100
print(f"-> {callable_pct:.1f}% de los bonos son callable")

# Gráfico 3: Callable
callable_analysis.plot(kind='pie', ax=axes[0,2], autopct='%1.1f%%',
                       colors=['#E91E63', '#9C27B0'],
                       labels=['Callable', 'No Callable'])
axes[0,2].set_title('Bonos Callable vs No Callable')
axes[0,2].set_ylabel('')

# Frecuencia de cupón
print("Frecuencia de Cupón:")
freq = universo['Coupon Frequency'].value_counts(dropna = False) 
for frequency, quantity in freq.items():
    print(f"  - {frequency} pago(s) por año: {quantity} bonos")

# Gráfico 4: Frecuencia
freq.plot(kind='bar', ax=axes[1,0], color='#00BCD4')
axes[1,0].set_title('Frecuencia de Pago de Cupón')
axes[1,0].set_xlabel('Pagos por año')
axes[1,0].set_ylabel('Número de Bonos')
axes[1,0].tick_params(axis='x', rotation=0)


# Maturity:
print("Bonos perpetuos o no (Maturity):")

maturity_date = universo[universo['Maturity'].notna() & (universo['Maturity'] != '')]
no_maturity_date= universo[universo['Maturity'].isna() | (universo['Maturity'] == '')]

print(f"  - Bonos con fecha de vencimiento: {len(maturity_date)}")
print(f"  - Bonos perpetuos (sin fecha): {len(no_maturity_date)}")

print("Para valorar los bonos perpetuos hay que tener en cuenta que la fecha de vencimiento es la de la primera call.")

# Gráfico 4: Maturity
labels = ['Con fecha de vencimiento', 'Perpetuos (sin fecha)']
sizes = [len(maturity_date), len(no_maturity_date)]
colors = ['#4CAF50', '#FFC107']

axes[1,1].pie(sizes, colors=colors, labels=labels, autopct='%1.1f%%', startangle=90)
axes[1,1].set_title('Distribución de Bonos por Maturity')

# Ocultar el grafico vacio
axes[1,2].axis('off')

plt.tight_layout()
plt.show()

#### <font color=#808080>Sectores y emisores</font>

In [ ]:
# Análisis de sectores y emisores
print("ANÁLISIS DE SECTORES Y EMISORES:")


# Sectores
sector_counts = universo['Industry Sector'].value_counts(dropna = False) 
print("Distribución por Sectores:")
for sector, quantity in sector_counts.items():
    print(f"  - {sector} : {quantity} bonos")
print(f"Total de sectores únicos: {universo['Industry Sector'].nunique()}")

# Emisores
emisor_counts = universo['Issuer'].value_counts(dropna = False) 
print(f"Total de emisores únicos: {universo['Issuer'].nunique()}")

print("Top 10 Emisores:")
top_emisors = emisor_counts.head(10)

for emisor, quantity in top_emisors.items():
    print(f"  - {emisor}: {quantity} bonos")

# Concentración
top_5_emisores_pct = top_emisors.head(5).sum() / len(universo) * 100
top_5_sectores_pct = sector_counts.head(5).sum() / len(universo) * 100

print("Análisis de Concentración:")
print(f"  - Top 5 emisores: {top_5_emisores_pct:.1f}% del universo")
print(f"  - Top 5 sectores: {top_5_sectores_pct:.1f}% del universo")

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Gráfico de sectores
sector_counts.plot(kind='barh', ax=axes[0], color=plt.cm.viridis(np.linspace(0, 1, len(sector_counts))))
axes[0].set_title('Distribución por Sectores')
axes[0].set_xlabel('Número de Bonos')

# Gráfico de concentración de emisores
emisor_counts.head(15).plot(kind='barh', ax=axes[1], color='#FF5722')
axes[1].set_title('Top 15 Emisores por Número de Bonos')
axes[1].set_xlabel('Número de Bonos')

plt.tight_layout()
plt.show()

**Conclusión**: No es una cartera diversificada, aunque se invirtiera en todos los bonos, porque 1012 bonos son del sector financiero de un total de 2255.

#### <font color=#808080>Ratings (Riesgo de crédito)</font>

In [ ]:
print("ANÁLISIS DE RATINGS:")

rating_order = [
    'AAA+', 'AAA', 'AAA-', 'AA+', 'AA', 'AA-', 'A+', 'A', 'A-',
    'BBB+', 'BBB', 'BBB-', 'BB+', 'BB', 'BB-', 'B+', 'B', 'B-',
    'CCC+', 'CCC', 'CCC-', 'CC', 'C', 'D', 'NR'
]

# Distribución de ratings
print("\n• Distribución de Ratings Específicos:")
rating_dist = universo['Rating'].value_counts(dropna = False)
for rating, quantity in rating_dist.items():
    print(f"  - {rating}: {int(quantity)} bonos")


print("\n• Resumen:")

rating_category_map = {}
for rating in rating_order:
    if rating == 'NR':
        rating_category_map[rating] = 'NR'
    elif rating_order.index(rating) <= rating_order.index('BBB-'):
        rating_category_map[rating] = 'IG'
    else:
        rating_category_map[rating] = 'HY'

# Mapear categorías desde rating_dist
category_counts = {'IG': 0, 'HY': 0, 'NR': 0}
for rating, count in rating_dist.items():
    categoria = rating_category_map.get(rating, 'NR')
    category_counts[categoria] += count

# Mostrar resumen
total_bonos = rating_dist.sum()

for cat in ['IG', 'HY', 'NR']:
    count = category_counts[cat]
    pct = count / total_bonos * 100
    print(f"  • {cat}: {int(count)} bonos ({pct:.1f}%)")

print("\n")

# Visualización
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Pie chart de categorías (IG, HY, NR)
category = ['IG', 'HY', 'NR']
porcentajes = [category_counts[cat] / total_bonos * 100 for cat in category]
axes[0].pie(
    porcentajes,
    labels=[f'{cat} - {pct:.1f}%' for cat, pct in zip(category, porcentajes)],
    autopct='%1.1f%%',
    startangle=90,
    colors=['#4CAF50', '#FF9800', '#9E9E9E']
)
axes[0].set_title('Distribución por Categoría de Rating')
axes[0].set_ylabel('')

# Bar chart de ratings específicos ordenados
rating_dist.plot(kind='bar', ax=axes[1], color='#3F51B5')
axes[1].set_title('Ratings Específicos (ordenados)')
axes[1].set_xlabel('Rating')
axes[1].set_ylabel('Número de Bonos')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


**Conclusión:** Hay un bono que tiene valor rating #N/A N/A. Consideramos que se trata un error de los datos ya que si no tuviese rating sería NR.

#### <font color=#808080>Riesgo de liquidez - Horquillas y nominal vivo</font>

In [ ]:
print("\n• Riesgo de liquidez:")

hor = universo[universo['Ask Price'] - universo['Bid Price'] > 0.5]

print(f"Los bonos que tienen riesgo de liquidez son: {len(hor)}")
hor.head(5)

In [ ]:
print("\n• Nominal vivo (Outstanding Amount):")
print("\n Los bonos con mayor nominal vivo suelen tener más negociación. Top 5: ")
oa = universo[["ISIN", "Issuer", "Outstanding Amount"]].sort_values(by="Outstanding Amount", ascending=False)
oa.head(5)

In [ ]:
print("\n Top 5 bonos con menor nominal vivo: ")
oa.tail(5)

#### <font color=#808080>Resto de información</font>

Otro datos que consideramos que se pueden analizar son PD 1YR.

In [ ]:
pd_1y = universo[['ISIN', 'Issuer', 'Rating', 'PD 1YR']]

pd_1y.sort_values(by="PD 1YR", ascending=False).head()

Destacamos que en el top cinco de bonos con mayor probabilidad de impago a un año, no aparece ninguno que no tenga rating.

In [ ]:
nr_pd_1y = universo[universo['Rating'] == 'NR'][['ISIN', 'Issuer', 'Rating', 'PD 1YR']]
nr_pd_1y.sort_values(by="PD 1YR", ascending=False).head()

In [ ]:
universo.isna().sum()

**GAPs de información:** 
Podemos ver que hay 19 bonos que no tienen Maturity ni Penultimate Coupon Date, son bonos perpetuos.
Hay 24 bonos que no tiene PD 1YR, no tenemos esa información. 
También podemos comprobar que de los 615 bonos que no tienen Next Call Date ninguno es callable.
Hay un bono que no tiene rating. Dada su baja probabilidad, 0,000436, estaría en un rating AAA+. Sin embargo, consideramos que no es bono válido. 

In [ ]:
len(universo[universo['Next Call Date'].isna() & universo['Callable'] == 'Y']['ISIN'])

In [ ]:
# Función para limpiar columnas numéricas
def clean_numeric_column(col):
    """Limpia columnas numéricas reemplazando valores no válidos"""
    if col.dtype == object:
        # Reemplazar comas por puntos como separador decimal si es necesario
        col = col.astype(str).str.replace(',', '.')
        # Reemplazar cadenas de texto comunes por NaN
        col = col.replace(['#N/D', '#N/A', 'N/A'], np.nan)
        # Convertir a numérico, forzando los errores a NaN
        col = pd.to_numeric(col, errors='coerce')
    return col

# Fecha de análisis
fecha_analisis = pd.to_datetime('2025-10-01')
print(f"Fecha de análisis: {fecha_analisis.strftime('%Y-%m-%d')}")


# Limpiar columnas numéricas
numeric_columns = ['Price', 'Coupon', 'PD 1YR', 'Outstanding Amount',
                   'Coupon Frequency', 'Bid Price', 'Ask Price']
for col in numeric_columns:
    if col in universo.columns:
        universo[col] = clean_numeric_column(universo[col])

# Convertir fechas
date_columns = ['Maturity', 'Next Call Date', 'First Coupon Date',
                'Penultimate Coupon Date', 'Issue date']
for columna in date_columns:
    if columna in universo.columns:
        universo[columna] = pd.to_datetime(universo[columna], format='%d/%m/%Y', errors='coerce')

# Calcular años hasta vencimiento
universo['years_to_maturity'] = (universo['Maturity'] - fecha_analisis).dt.days / 365.25

print(f"\n nº fondos: {len(universo)} bonos")
print(" Información general del universo:")
print(universo.head())

universo.info()

En esta sección, analizamos el resto de ficheros para ver qué información tenemos y, en caso de haber *gaps*, limpiar los datos antes de trabajar con ellos.

##### <font color=#CC6600>Precios bonos universo</font>

In [ ]:
precios_hist_universo_path = Path('../data/precios_historicos_universo.csv')
precios_historicos_universo = pd.read_csv(precios_hist_universo_path, sep=';', encoding='utf-8-sig', header=None, dtype='str')

In [ ]:
precios = precios_historicos_universo.drop(precios_historicos_universo.columns[1], axis=1)

Se elimina la columna con fecha, 2023-10-02, ya que es domingo y no se tiene datos anteriores.

In [ ]:
precios.iloc[0,0] = 'ISIN'
fechas = precios.iloc[0, 1:]
columna_isin = precios.iloc[1:, 0]

In [ ]:
precios_nd = precios.iloc[1:,1:].copy()
precios_nd.replace('#N/D', pd.NA, inplace = True)
precios_nan = precios_nd.apply(pd.to_numeric, errors='coerce')

precios_ffill = precios_nan.ffill(axis=1)

Se sustituye #N/D por NaN, y se rellenan los datos con los precios del dia anterior en caso de que sea fin de semana. 

In [ ]:
fila_fechas = pd.DataFrame([['ISIN'] + list(fechas)], columns=precios.columns)
data = pd.concat([columna_isin.reset_index(drop=True), precios_ffill.reset_index(drop=True)], axis=1)
data.columns = precios.columns

In [ ]:
final_precios_hist_univ = pd.concat([fila_fechas, data], ignore_index=True)

##### <font color=#CC6600>Otros precios</font>

Para terminar con el análisis de datos, falta lo leído en los ficheros de *"precios_historicos_varios.csv"* y *curvaESTR.csv*.

In [ ]:
precios_hist_varios_path = Path('../data/precios_historicos_varios.csv')
precios_historicos_varios = pd.read_csv(precios_hist_varios_path, sep=';', encoding='utf-8-sig')

In [ ]:
precios_historicos_varios.columns.values[0] = 'Fecha'
precios_historicos_varios.head()

En el archivo de precios_historicos_varios, se encuentra los datos de cinco indices con el rango de fechas hasta 2023-10-02. Los indices son:  
* ITRX EUR CDSI GEN 5Y Corp,  iTraxx Europe Main 5Y:  representa un índice de Credit Default Swaps (CDS) sobre 125 empresas europeas con calificación de crédito investment grade, con horizonte en 5 años. 
* ITRX XOVER CDSI GEN 5Y Corp: Índice de Credit Default Swaps (CDS) con horizonte en 5 años, 75 empresas europeas con calificación por debajo de grado de inversión, high yield.
* DU1 Comdty, Euro-Bund Futures: su subyacente es el bono del gobierno alemán con vencimiento a 10 años.
* OE1 Comdty, Euro-OAT Futures: su subyacente es el bono del gobierno francés y con vencimiento a 5 años.
* RX1 Comdty, Euro-Buxl Futures: su subyacente es el bono del gobierno aleman con vencimiente de 24 1a 35 años. 
* RECMTREU Index, Bloomberg MSCI Euro Corporate SRI Total Return Index Unhedged: Indice de renta fija corporativa.

In [ ]:
curva_estr_path = Path('../data/curvaESTR.csv')
curva_estr = pd.read_csv(curva_estr_path, sep=';', encoding='utf-8-sig')

In [ ]:
curva_estr.head(10)

In [ ]:
#Convertir fechas a días desde hoy
curva_estr['Date'] = pd.to_datetime(curva_estr['Date'], dayfirst=True) #columna Date en formato datetime
fecha_valor = pd.to_datetime('10/10/2025', dayfirst=True) #define la fecha de valoración
curva_estr['Days'] = (curva_estr['Date'] - fecha_valor).dt.days #Calcula los dias desde la fecha de valoracion
curva_estr['YearFrac'] = curva_estr['Days'] / 365  #Calcula la fraccion de año con base ACT/365

In [ ]:
#La columna Zero Rate = Tasa Spot

plt.figure(figsize=(10, 5))
plt.plot(curva_estr['YearFrac'], curva_estr['Zero Rate'], marker='o', linestyle='-', color='blue')
plt.title('Curva ESTR (Tasa Spot)')
plt.xlabel('Fracción de Año desde 10/10/2025')
plt.ylabel('Tasa Spot (%)')
plt.grid(True)
plt.tight_layout()
plt.show()


### <font color=#336699>2. Valoración</font>

<style>.white {background-color: #595959}

</style><div class="gray">

❕💬 En esta sección, valoraremos los bonos utilizando la curva. Para ello, crea una función (puedes hacerlo en un .py aparte) que con las **características del bono, la curva y un spread de crédito** devuelva la valoración del bono (incluyendo **precio limpio, cupón corrido y precio sucio**).

Si asumimos que el **spread de crédito es 0**, y la ejecutamos para el 01/10/2025...
- ¿Qué observas si comparas los precios obtenidos y los precios de mercado?
- ¿Crees que la diferencia se debe a un factor relacionado sólo con el riesgo crediticio?
- ¿Qué otros factores influyen en ese spread?

Para la valoración, haz las siguientes simplificaciones:

- Asume que el vencimiento de los bonos perpetuos (para los que no hay vencimiento) es la próxima fecha call.
- Asume que todos aquellos bonos que tengan call serán calleados. Por lo tanto, usa la fecha call como fecha de vencimiento.
- Asume que los cupones son fijos hasta vencimiento (aunque alguno cambie a lo largo de la vida del bono).
- Usa la base de cálculo ACT/365. No tengas en cuenta la convención de día hábil.

Ten en cuenta que necesitarás una función de interpolación también. Interpola los factores de descuento exponencialmente.

</div>

In [ ]:
# Establecemos la fecha
fecha_valor = datetime.datetime(2025,10,1)

In [ ]:
import simulation

curva = simulation.read_curve('curvaESTR.csv')
# Calcular los precios limpios, sucios y cupón corrido
universo_evaluado = simulation.evaluate_bonds(fecha_valor, universo, curva, 0)
universo_evaluado.head()


A continuación se muestra una comparativa de los precios limpios estimados (los que ven en herramientas como Bloomberg) con los del mercado. Como los precios históricos cubren hasta el 01/10/2025, compararemos el precio que hemos calculado con fecha del 01/10/2025 con los precios históricos del mercado en ese mismo día.

In [ ]:
precios_hist_universo_path = Path('../data/precios_historicos_universo.csv')
precios_historicos_universo = pd.read_csv(precios_hist_universo_path, sep=';', encoding='utf-8-sig', dtype='str')
precios_historicos_universo.head()
columna_fecha_hoy = fecha_valor.strftime('%d/%m/%Y')
precios_historicos_universo = precios_historicos_universo.rename(columns={precios_historicos_universo.columns[0]: "ISIN"})
precios_historicos_universo = precios_historicos_universo.rename(columns={columna_fecha_hoy: "Market Price"})
# Eliminamos 'CORP' del ISIN
precios_historicos_universo['ISIN'] = precios_historicos_universo['ISIN'].apply(lambda i: i.split(' ')[0])

universo_precios_historicos = pd.merge(universo_evaluado, precios_historicos_universo[['ISIN', 'Market Price']], on='ISIN')
# Como hemos leido el csv como str, debemos pasar la columna de precios a float
universo_precios_historicos['Market Price'] = pd.to_numeric(universo_precios_historicos['Market Price'], errors='coerce')
universo_precios_historicos['% Error'] = abs(universo_precios_historicos['Market Price'] - universo_precios_historicos['Clean Price']) / universo_precios_historicos['Market Price'] * 100
universo_precios_historicos.head(10)

Al comparar los precios calculados con los de mercados vemos que los porcentajes de error son bastante elevados. Aunque también influye utilizar un spread = 0 en el cálculo, no es el único factor que causa esta diferencia; se deben tener en cuenta factores como la oferta/demanda de cada bono, el rating del issuer, nominal vivo, etc. Es decir, factores no contemplados que hacen más o menos atractivo invertir en un bono frente a otros.

### <font color=#336699>3. Spread</font>

<style>.white{background-color: #595959}

</style><div class="gray">

❕💬 Calculemos ahora los spreads que debemos añadir a la curva con un movimiento paralelo para que cuadren los precios de mercado que tenemos. Para ello, usa la función de valoración del apartado anterior.
- ¿Qué observas? ¿Tienen sentido los resultados?
- ¿Con qué datos de los que tenemos compararías para ver si los resultados son coherentes?

</div>

In [ ]:
universo_z_spread = simulation.calculate_z_spreads(fecha_valor, universo_precios_historicos, curva)


In [ ]:
plt.figure(figsize=(16, 10))

# Definimos el orden (el mismo de antes)
rating_order = [
    'AAA', 'AA+', 'AA', 'AA-', 'A+', 'A', 'A-',
    'BBB+', 'BBB', 'BBB-', 'BB+', 'BB', 'BB-', 'B+', 'B', 'B-'
]

# Eliminamos los datos con más de 1500 puntos básicos
df_plot_clean = universo_z_spread[universo_z_spread['Z Spread'] < 1500].copy()

# Lo hacemos más transparente (alpha=0.5) para que no tape los puntos
sns.boxplot(
    x='Rating', 
    y='Z Spread', 
    data=df_plot_clean,
    order=rating_order,
    boxprops=dict(alpha=0.5),
    showfliers=False
)

# Pone un punto por cada bono real
sns.stripplot(
    x='Rating', 
    y='Z Spread', 
    data=df_plot_clean, 
    order=rating_order, 
    color='black',
    alpha=0.6,
    jitter=0.2,
    size=4
)

plt.title('Análisis Detallado del Z-Spread: Distribución y Datos Individuales')
plt.ylabel('Z-Spread (puntos básicos)')
plt.xlabel('Calidad Crediticia (Rating)')
plt.grid(True, alpha=0.3, axis='y')

# Añadimos una línea roja en 0 como referencia
plt.axhline(y=0, color='r', linestyle='--', linewidth=1)

plt.show()

**Conclusión:** Los bonos a los que hay que aplicar un spread negativo son bonos que están más caros de lo que se esperaba con la curva de Discount Rate. Esto implicaría que se espera que el Euro no se deprecie tanto en el futuro.

In [ ]:
# 1. Preparar datos para la gráfica
# Filtramos outliers extremos (ruido) para que se vea bien
df_scatter = universo_z_spread[(universo_z_spread['Z Spread'] > -2000) & (universo_z_spread['Z Spread'] < 2000)].copy()

# Creamos una etiqueta para colorear
df_scatter['Tipo Spread'] = df_scatter['Z Spread'].apply(lambda x: 'Negativo' if x < 0 else 'Positivo')

# 2. Generar Gráfica
plt.figure(figsize=(14, 8))
sns.scatterplot(
    data=df_scatter,
    x='Ask Price',
    y='Z Spread',
    hue='Tipo Spread',
    palette={'Negativo': 'red', 'Positivo': 'blue'},
    alpha=0.6
)

# Líneas de referencia
plt.axhline(0, color='black', linestyle='--', label='Spread Cero')
plt.axvline(100, color='green', linestyle=':', label='Precio a la Par (100)')

plt.title('El "Efecto Call": Precios Altos provocan Spreads Negativos')
plt.xlabel('Precio de Mercado (Ask Price)')
plt.ylabel('Z-Spread (bps)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### <font color=#336699>4. YTM, Duración, Convexidad</font>

<style>.white {background-color: #595959}

</style><div class="gray">

❕💬 Calculemos ahora la siguiente información, también relacionada con la rentabilidad y riesgo de las emisiones:
- *Yield* - Por simplicidad, en este caso, en el caso de los bonos callable, nos quedaremos con la fecha call, como en el ejercicio anterior. Usa las mismas asunciones que para la valoración y el spread.
- Duración
- Convexidad

Responde a las siguientes preguntas:
- ¿Que relación hay entre la TIR calculada y el spread calculado en el apartado anterior?
- ¿Qué relación hay entre la duración y el vencimiento? ¿Qué refleja la duración? ¿De qué otra forma se podría obtener esta sensibilidad?
- Estima el precio del bono usando la duración y convexidad, ¿qué observas?

</div>

In [ ]:
universo_ytm = simulation.calculate_ytms(fecha_valor, universo_z_spread)
universo_ytm.head()


Teoricamente el valor de la TIR es aproximadamente a la curva de Zero Rate + Spread.

In [ ]:
universo_duration = simulation.calculate_durations(fecha_valor, universo_ytm)
universo_duration.head()

Siendo el vencimiento la fecha en la que se devuelve el principal y la duración es el plazo medio ponderado en el que se reciben los flujos de caja, cupones + principal. Se puede concluir:
* El bono cupón cero tiene una duración igual a la fecha de vencimiento, ya que todos los flujos se reciben al final.
* En un bono de cupones, la duración es menor a la fecha de vencimiento, porque se reciben los cupones poco a poco.
* Cuanto mayor sea el cupón, menor será la duración en comparación con el vencimiento.

In [ ]:
universo_convexity = simulation.calculate_convexities(fecha_valor, universo_ytm)
universo_convexity['Bond Duration (~ years)'] = universo_convexity['Maturity'].apply(lambda md: (md - fecha_valor).days / 365)
convexities = universo_convexity[['ISIN', 'Bond Duration (~ years)', 'Convexity']]

In [ ]:
convexities[convexities['Bond Duration (~ years)'] < 3].sort_values(by='Convexity').head()

Los anteriores son los bonos que menos queda para liquidarse, por lo son bonos poco sensibles al cambio de tipos. Prácticamente, no hay curvatura significativa en la relación precio-YTM.

In [ ]:
convexities[(convexities['Bond Duration (~ years)'] >= 5) & (convexities['Bond Duration (~ years)'] < 10)].sort_values(by='Convexity', ascending=False).head()

Los bonos con una duración a medio-largo plazo, emisores de 10 años. La curva ya se puede apreciar, pero no es exagerada.

In [ ]:
convexities[(convexities['Bond Duration (~ years)'] >= 15) & (convexities['Bond Duration (~ years)']) < 30].sort_values(by='Convexity', ascending=False).head()

Como se ve en la tabla, los valores son más altos cuando se trata de bonos de duraciones más largas. Las duraciones altas suelen implicar mucha sensibilidad a tipos. Y estos valores reflejan esa curvatura adicional.

In [ ]:
estimacion_aproximada = universo_convexity['Price'] * (-universo_duration['Duration_Modificada'])* universo_duration['YTM']/100
estimacion_aproximada.name= 'estimacion_aproximada'
estimacion_aproximada.head()

La estimación aproximada es el cambio de precio de un bono ante una variación de yield. Estos resultados muestran lo que caería cada bono en el supuesto de aumento de tipos.

### <font color=#336699>5. Cartera equiponderada</font>

<style>.white{background-color: #595959}

</style><div class="gray">

❕💬 Crea un algoritmo de inversión que consista en una cartera equiponderada, asignando el mismo peso a todos los bonos vivos en cada fecha de rebalanceo.

Asume rebalanceo mensual, y que no tenemos costes más allá de los implícitos en el propio precio, calcula la evolución que hubiese tenido tu algoritmo. Por simplificación, utiliza los precios MID que se te dan.

Asumiendo que el benchmark de la cartera es el índice que se nos da: *RECMTREU Index*. Contrasta la evolución de tú cartera contra dicho benchmark. Ten cuidado porque es un índice *Total Return*.

- ¿Qué sería lo más correcto en lugar de utilizar los precios MID?
- ¿Se te ocurre algún otro benchmark que se podría utilizar?

</div>

In [ ]:
#  CARTERA EQUIPONDERADA CON REBALANCEO MENSUAL
fechas_rebalanceo = pd.date_range(precios_historicos_varios['Fecha'].min(), fecha_analisis, freq='ME')

# Preparar datos: ISINs como índice, fechas como columnas
precios_df = final_precios_hist_univ.iloc[1:].set_index(0)
precios_df.columns = pd.to_datetime(final_precios_hist_univ.iloc[0, 1:], dayfirst=True)
precios_df = precios_df.apply(pd.to_numeric, errors='coerce')
precios_df.index = precios_df.index.str.split(' ').str[0]

# Benchmark RECMTREU
benchmark = precios_historicos_varios.set_index(pd.to_datetime(precios_historicos_varios['Fecha'], dayfirst=True))['RECMTREU Index'].astype(float)

# Calcular retornos equiponderados CON CARRY
retornos, fechas = [], []
universo_precios = universo[['Maturity', 'Next Call Date', 'ISIN', 'Coupon']].join(precios_df, on='ISIN')
for i, fecha in enumerate(fechas_rebalanceo[1:], 1):
    fecha_ant = fechas_rebalanceo[i-1]
    dias = (fecha - fecha_ant).days # calculamos los dias transcurridos
    cols_act = fecha
    cols_ant = fecha_ant
    if cols_ant is None or cols_act is None: continue

    universo_precios = universo_precios[(universo_precios['Maturity'] > fecha) & (universo_precios['Next Call Date'].isna() | (universo_precios['Next Call Date'] > fecha))]
    p_ant = universo_precios[['ISIN', 'Coupon', cols_ant]]
    p_act = universo_precios[['ISIN', 'Coupon', cols_act]]

    comunes = p_ant.merge(p_act, on=['ISIN', 'Coupon'])
    if len(comunes) == 0: continue
    ret_precio = (comunes[cols_act]/comunes[cols_ant]).apply(lambda val: val - 1).mean()
    carry = comunes['Coupon'].mean() / 100 * (dias / 365)   # calculamos el carry
    retornos.append(ret_precio + carry) # sumamos el carry al retorno

    fechas.append(fecha)

# Acumular retornos
cartera_acum = (1 + pd.Series(retornos, index=fechas)).cumprod()
cartera_acum /= cartera_acum.iloc[0]
benchmark_norm = benchmark.resample('ME').last().loc[fechas[0]:fechas[-1]]
benchmark_norm /= benchmark_norm.iloc[0]

print(f"Retorno cartera: {(cartera_acum.iloc[-1]-1)*100:.2f}% | Benchmark: {(benchmark_norm.iloc[-1]-1)*100:.2f}%")
print("\n No tenemos en cuenta la diferencia entre bid/ask, para un mayor realismo deberíamos simular que siempre se compra el precio bid y se vende el precio ask.")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(cartera_acum, 'b-', lw=2, label='Cartera Equiponderada')
plt.plot(benchmark_norm, 'r--', lw=2, label='RECMTREU (Benchmark)')
plt.axhline(1, color='gray', ls=':', alpha=0.5)
plt.xlabel('Fecha'), plt.ylabel('Valor Normalizado'), plt.title('Cartera vs Benchmark')
plt.legend(), plt.grid(alpha=0.3), plt.tight_layout(), plt.show()

### <font color=#336699>6. Cartera mandato</font>

<style>.white {background-color: #595959}

</style><div class="gray">

❕💬 Como adelantábamos en el enunciado, tienes el mandato de construir una cartera de como máximo **20** bonos corporativos con ese universo y una serie de restricciones y, claro, maximizando la rentabilidad total de la cartera:
- La duración de la cartera no debe superar los 3 años
- La exposición a emisiones HY no puede superar el 10% de la cartera
- No puedes invertir en deuda subordinada
- No se puede invertir en emisiones de tamaño igual o inferior a 500 millones
- No se puede invertir más de un 10% del capital en una misma emisión
- No puede haber más de un 15% de concentración en un mismo emisor
(¡OJO! No estamos teniendo en cuenta en este ejercicio si hubiera un mínimo de inversión, lo cuál sería un dato relevante tener en cuenta en un caso real)

1. Teniendo en cuenta la naturaleza que nos están pidiendo para la cartera, ¿añadirías alguna otra restricción?

2. ¿Cómo medirías el riesgo de crédito de la cartera?

3. ¿Cómo medirías el riesgo de liquidez de la cartera? ¿Se te ocurre alguna otra información que se podría utilizar aunque no se te haya dado?

4. Describe cómo habría que hacer el backtest de esta cartera, no hace falta que lo implementes en este caso

</div>

In [ ]:
from dateutil.relativedelta import relativedelta

# 1. Pre-filtrar universo según restricciones fijas
ratings_hy = ['BB+', 'BB', 'BB-', 'B+', 'B', 'B-', 'CCC+', 'CCC', 'CCC-', 'CC', 'C', 'D']
df = universo_duration[
    (universo_duration['Maturity'] <= (fecha_analisis + relativedelta(years=3))) &
    (universo_duration['Outstanding Amount'] > 500_000_000) &  # nos piden que no invirtamos en los bonos de menos de 500M
    (~universo_duration['Seniority'].str.contains('Subordinated', case=False, na=False))  # elimina deuda subordinada, solo senior
].copy()
df['is_HY'] = df['Rating'].isin(ratings_hy).astype(int)# introducimos una marca a los bonos HY

# 2. Matrices para optimización
n = len(df)
ytm = df['YTM'].values
dur = df['Duration_Modificada'].values
hy = df['is_HY'].values
emisores = pd.get_dummies(df['Issuer'])  # Matriz de emisores para calcular peso por emisor

# 3. Función objetivo: maximizar YTM = minimizar -YTM
def objetivo(w): return -np.dot(w, ytm)

# 4. Restricciones
constraints = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1},                    # Suma pesos = 1
    {'type': 'ineq', 'fun': lambda w: 0.10 - np.dot(w, hy)},           # HY <= 10%
] + [{'type': 'ineq', 'fun': lambda w, e=e: 0.15 - np.dot(w, emisores[e])} for e in emisores.columns]  # Emisor <= 15%

bounds = [(0, 0.10) for _ in range(n)]  # 0 <= w_i <= 10% límites por bono(restricción)

# 5. Optimizador SLSQP
w0 = np.ones(n) / n
result = minimize(objetivo, w0, method='SLSQP', bounds=bounds, constraints=constraints)
df['Peso'] = result.x

# 6. Seleccionar top 20 bonos con mayor peso
cartera = df[df['Peso'] > 0.001].nlargest(20, 'Peso')
cartera['Peso'] = cartera['Peso'] / cartera['Peso'].sum()  # Renormalizar

# 7. Métricas de la cartera
duracion_cartera = (cartera['Peso'] * cartera['Duration_Modificada']).sum()
print(f"Bonos en cartera: {len(cartera)}")
print(f"YTM cartera: {(cartera['Peso'] * cartera['YTM']).sum():.2f}%")
print(f"Duración cartera: {duracion_cartera:.2f} años")
print(f"Exposición HY: {(cartera['Peso'] * cartera['is_HY']).sum()*100:.2f}%")
cartera[['ISIN', 'Issuer', 'Rating', 'YTM', 'Duration_Modificada', 'Peso']].round(4)

##### <font color=#CC6600>Riesgo de crédito</font>

In [ ]:
cartera['P Defaulting'] = (1 - (1 - cartera['PD 1YR'])**cartera['Duration_Modificada'])*100
cartera

Aunque encontremos valores de *P Defaulting* = NaN, no podemos sustituirlos por 0, ya que esto sería asumir que un dato que desconocemos es decir que no hay probabilidad de impago.

In [ ]:
cartera['EAD'] = cartera.apply(lambda bond: simulation.get_accrued_interest(bond, fecha_valor), axis = 1)
cartera

Al no disponer de datos de recuperación no es posible calcular correctamente el LGD, por lo que se utilizará el estándar Basel (LGD = 40%)

In [ ]:
cartera['LGD'] = 0.4
cartera

In [ ]:
cartera['Expected Loss'] = cartera['EAD'] * cartera['P Defaulting']/100 * cartera['LGD']
cartera

##### <font color=#CC6600>Riesgo de liquidez</font>

In [ ]:
aprox_volume = cartera['Outstanding Amount'] * cartera['Bid Price'] * 1/100
cartera['Riesgo Liquidez'] = (cartera['Bid Price'] - cartera['Ask Price'])/aprox_volume/cartera['Outstanding Amount']
cartera

In [ ]:
numerador = (cartera['Ask Price'] - cartera['Bid Price'])
denominador = (cartera['Bid Price'] + cartera['Ask Price'])/2
cartera['Riesgo Liquidez'] = (numerador/denominador)*10000
cartera

##### <font color=#CC6600>Backtest</font>

La estrategía de backtesting que utilizaría estaría compuesta por varios tipos de estrategias: como por ejemplo el backtesting historico, stress testing y testing por segmentos.

En cuanto al backtesting, la estrategía sería usar datos de precios del pasado, spreads de crédito y horquillas de liquidez, para poder evaluar si el modelo que se ha desarrollado subestima o sobreestima la volatilidad y el deterioro crediticio.

En cuanto al stress testing, se podría analizar como se comporta la cartera creada cuando hay crisis de liquidez, o si hubiera aumentos bruscos de default en algún o algunos sectores, o cambios en el rating de las empresas. Así podríamos saber como se comportaría la cartera bajo estas condiciones.

Si estudiasemos el testing por segmentos, se podría dividir la cartera por segmentos, comparando el grado de inversión con el high yield, con los sectores o con los emisores. Con la finalidad de poder detectar la falta de diversificación de la cartera.

En conclusión, una estrategía de backtesting combinando varias opciones nos daría una visión más realista y completa del comportamiento de nuestra cartera basandonos en datos historicos.



### <font color=#336699>7. Cobertura tipos de interés</font>

<style>.white {background-color: #595959}

</style><div class="gray">

❕💬 Utiliza alguno de los siguientes instrumentos de los que te hemos dado para cubrir la duración (sensibilidad de tipos de interés) de la cartera que has construido según el mandato. Asume una inversión en la cartera de 10 millones:

- Futuros sobre el *Schatz* (ticker: DU1) - Duración a 01/10/2025: 1.92
- Futuros sobre el *BOBL* (ticker: OE1) - Duración a 01/10/2025: 5.44
- Futuros sobre el *BUND* (ticker: RX1) - Duración a 01/10/2025: 10

*Contract size* en todos los casos: 100,000 euros

Investiga sobre estos instrumentos antes de tomar la decisión. Razona tu elección del instrumento y el número de contratos que has decidido comprar/vender.

- ¿Qué pasaría si comprásemos/vendiésemos 100 futuros?
- ¿Se te ocurre algún otro instrumento con el que cubrir la sensibilidad a los tipos de interés de la cartera?

</div>

In [ ]:
# COBERTURA TIPOS DE INTERÉS

#Si tenemos una posición grande y los tipos suben el valor de nuestros bonos baja
inversion = 10000000
dur_cartera = ((cartera['Peso'] * cartera['Duration_Modificada']).sum().real)  # Duración de cartera del apartado 6
contract_size = 100000

# Futuros disponibles (duración a 01/10/2025)
futuros = {'DU1 (Schatz)': 1.92, 'OE1 (BOBL)': 5.44, 'RX1 (BUND)': 10}

# Calcular contratos necesarios: N = (Dur_cartera × Valor_cartera) / (Dur_futuro × Valor_futuro)
for nombre, dur_fut in futuros.items():
    n_contratos = (dur_cartera * inversion) / (dur_fut * contract_size)
    print(f"{nombre}: VENDER {n_contratos:.0f} contratos")




In [ ]:
# QUÉ PASA CON 100 FUTUROS?
for nombre, dur_fut in futuros.items():
    dur_cubierta = (100 * contract_size * dur_fut) / inversion
    dur_resultante = dur_cartera - dur_cubierta
    estado = "sub-cubierto" if dur_resultante > 0 else ("sobre-cubierto " if dur_resultante < 0 else "perfecto ✓")
    print(f"100 {nombre}: Duración resultante = {dur_resultante:.2f} años → {estado}")


### <font color=#336699>8. Cobertura crédito</font>

<style>.white{background-color: #595959}

</style><div class="gray">

❕💬 Utiliza alguno de los siguientes instrumentos de los que te hemos dado para cubrir el riesgo de crédito de la cartera que has construido según el mandato. Asume una inversión en la cartera de 10 millones:

- ITRAXX Main (ticker: ITRX EUR CDSI GEN 5Y Corp)
- ITRAXX XOVER (ticker: ITRX XOVER CDSI GEN 5Y Corp)

Estos índices cotizan en forma de spread, en puntos básicos. La sensibilidad del valor del swap (CDS) la vamos a asumir en 4,500€ al punto básico asumiendo una inversión de 10 millones.

Investiga sobre estos instrumentos antes de tomar la decisión. Razona tu elección del instrumento y el nominal que has decidido comprar/vender.

- ¿Tiene sentido plantear esta cobertura total?
- ¿Con qué otros instrumentos podrías cubrir el riesgo de crédito?

</div>

In [ ]:
# COBERTURA RIESGO DE CRÉDITO

#Si es spread empeora, como por ejemplo si cambia la calidad crediticia de la cartera el valor cambia. Aquí nos cubrimos ese riesgo de que el spread empeore.

inversion = 10000000
sensibilidad_cds = 4500  # €/bp para 10M

# Spread ponderado de la cartera
spread_cartera = float((cartera['Peso'] * cartera['Z Spread']).sum())
exposicion_hy = float((cartera['Peso'] * cartera['is_HY']).sum())

# Índices disponibles (forzar a float)
itraxx = {
    'ITRAXX Main (IG)': float(precios_historicos_varios['ITRX EUR CDSI GEN 5Y Corp'].iloc[-1]),
    'ITRAXX XOVER (HY)': float(precios_historicos_varios['ITRX XOVER CDSI GEN 5Y Corp'].iloc[-1])
}

# Elección: depende de la exposición a bonos HY, si es mayor de 50% se elige el ITRAXX XOVER (HY), si es menor se elige el ITRAXX Main (IG)
indice_elegido = 'ITRAXX Main (IG)' if exposicion_hy < 0.5 else 'ITRAXX XOVER (HY)'
spread_indice = itraxx[indice_elegido]

# Nominal a cubrir
ratio_cobertura = spread_cartera / spread_indice
nominal_cds = inversion * ratio_cobertura

print(f"El spread medio ponderado de la cartera: {spread_cartera:.0f} bps sobre la curva libre de riesgo ")
print(f"El spread del {indice_elegido}: {spread_indice:.0f} bps")
print(f"Exposición HY: {exposicion_hy*100:.1f}%")
print(f"\n→ Instrumento: {indice_elegido}")
print(f"→ COMPRAR protección por nominal: {nominal_cds:,.0f}€")
print(f"→ Sensibilidad cobertura: {sensibilidad_cds * ratio_cobertura:,.0f}€/bp")

In [ ]:
#  ¿COBERTURA TOTAL?
# Si cubrimos 100% del nominal
cobertura_total = sensibilidad_cds  # €/bp
perdida_cartera_10bp = inversion * (spread_cartera/10000) * 0.001 * 10  # Aproximación
print(f"Cobertura total (10M nominal): {cobertura_total:,}€/bp")
print(f"\n¿Tiene sentido? Depende:")
print("- Cobertura total elimina riesgo pero también el carry (YTM)")
print("- Cobertura parcial permite mantener parte del rendimiento")
print("- Coste del CDS reduce rentabilidad neta")

print("\n¿Otros instrumentos para cubrir crédito?")
print("  • CDS individuales sobre cada emisor.")
print("  • Vender bonos corporativos y comprar bonos gobierno.")
print("  • ETFs inversos de crédito.")
print("  • Opciones sobre ITRAXX.")

### <font color=#336699>9. Estrategia propia</font>

<style>.white {background-color: #595959}

</style><div class="gray">

❕💬 Plantea tu propia estrategia con la información que tienes. Puede ser una estrategia direccional, de valor relativo, que hayas visto o no en clase; pero siempre razonando tu planteamiento.

</div>

In [ ]:
# APARTADO 9: ESTRATEGIA VALOR RELATIVO (Z-SPREAD vs RATING)
#Propuesta: Comprar (posiciones largas) bonos "baratos" (con spread alto para su rating) y vender posiciones cortas "caros" (spread bajo para su rating)

# 1. Calcular spread medio por rating
df = universo_duration[universo_duration['Z Spread'] > 0].copy()
spread_medio_rating = df.groupby('Rating')['Z Spread'].mean()
df['Spread_Esperado'] = df['Rating'].map(spread_medio_rating)
df['Spread_Diff'] = df['Z Spread'] - df['Spread_Esperado']  # Positivo = barato, Negativo = caro

# 2. Seleccionar top 10 baratos (LONG) y top 10 caros (SHORT)
long_bonds = df.nlargest(10, 'Spread_Diff')[['ISIN', 'Issuer', 'Rating', 'Z Spread', 'Spread_Diff', 'Duration_Modificada']]
short_bonds = df.nsmallest(10, 'Spread_Diff')[['ISIN', 'Issuer', 'Rating', 'Z Spread', 'Spread_Diff', 'Duration_Modificada']]

# 3. Duration-neutral: ajustar pesos para igualar duración long = duración short
dur_long, dur_short = long_bonds['Duration_Modificada'].mean(), short_bonds['Duration_Modificada'].mean()
ratio = dur_long / dur_short  # Ajuste para neutralizar duración

print("=== ESTRATEGIA VALOR RELATIVO ===")
print(f"\nLONG (bonos baratos) - Duración media: {dur_long:.2f}")
print(long_bonds.to_string(index=False))
print(f"\nSHORT (bonos caros) - Duración media: {dur_short:.2f}")
print(short_bonds.to_string(index=False))
print(f"\n→ Ratio cobertura: {ratio:.2f}x nominal en SHORT para neutralizar duración")

In [ ]:
# IMPLEMENTACIÓN CON 10M
inversion = 10_000_000
nominal_long = inversion / 2
nominal_short = float((inversion / 2) * ratio ) # Ajustado por duración

# Rendimiento esperado: convergencia de spreads al valor justo
beneficio_esperado_bp = df.loc[long_bonds.index, 'Spread_Diff'].mean() - df.loc[short_bonds.index, 'Spread_Diff'].mean()
beneficio_eur = beneficio_esperado_bp * (nominal_long / 10000)  # Aproximación

print(f"Nominal LONG: {nominal_long:,.0f}€")
print(f"Nominal SHORT: {nominal_short:,.0f}€")
print(f"Diferencial spreads: {beneficio_esperado_bp:.0f} bps")
print(f"Beneficio potencial (convergencia): {beneficio_eur:,.0f}€")

# Cobertura de tipos (apartado 7)
dur_neta = dur_long - dur_short * ratio
print(f"\nDuración neta: {dur_neta:.2f} (≈ 0 si está bien cubierta)")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# LONG (baratos)
axes[0].barh(long_bonds['Issuer'], long_bonds['Spread_Diff'], color='green', alpha=0.7)
axes[0].set_xlabel('Spread Diff (bps)'), axes[0].set_title('LONG - Bonos Baratos (Spread > Esperado)')
axes[0].axvline(0, color='black', linestyle='--', alpha=0.5)

# SHORT (caros)
axes[1].barh(short_bonds['Issuer'], short_bonds['Spread_Diff'], color='red', alpha=0.7)
axes[1].set_xlabel('Spread Diff (bps)'), axes[1].set_title('SHORT - Bonos Caros (Spread < Esperado)')
axes[1].axvline(0, color='black', linestyle='--', alpha=0.5)

plt.tight_layout(), plt.show()